## Обучающий курс по библиотеке Requests в Python

### Введение
Библиотека requests - это одна из самых популярных и удобных библиотек Python для работы с HTTP-запросами. Она предоставляет простой и элегантный способ взаимодействия с веб-сервисами.

In [1]:
import requests
import json
from requests.exceptions import RequestException
from concurrent.futures import ThreadPoolExecutor
import time
import os

### Модуль 1: Основы работы с Requests

In [ ]:
# Простой GET-запрос
response = requests.get('https://api.github.com')

# Проверка статуса ответа
print(f"Status Code: {response.status_code}")  # 200 - успешно

# Получение содержимого ответа
print(f"Content: {response.text}")  # Первые 100 символов

# Получение ответа в формате JSON (если ответ JSON)
if response.headers.get('Content-Type') == 'application/json':
    data = response.json()
    print(f"JSON Data: {data}")

In [ ]:
# Параметры запроса
params = {
    'tab': 'projects'
}

response = requests.get('https://github.com/mrvdanilov/', params=params)
print(f"URL: {response.url}")  # Показывает итоговый URL с параметрами

### Модуль 2: Обработка ответов

In [ ]:
response = requests.get('https://httpbin.org/get')

print(f"Status Code: {response.status_code}")
print(f"Headers: {dict(response.headers)}")
print(f"Content Type: {response.headers.get('Content-Type')}")
print(f"Encoding: {response.encoding}")
print(f"Response Time: {response.elapsed.total_seconds()} seconds")

# Проверка успешности запроса
if response.status_code == 200:
    print("Запрос успешен!")
else:
    print(f"Ошибка: {response.status_code}")

In [ ]:
response = requests.get('https://api.github.com/users/octocat')

# Для JSON ответов
if 'application/json' in response.headers.get('Content-Type', ''):
    user_data = response.json()
    print(f"User: {user_data.get('login')}")
    print(f"Name: {user_data.get('name')}")

# Для текстовых ответов
else:
    print(response.text)


In [ ]:
# Сохранение бинарных данных (например, изображение)
image_response = requests.get('https://httpbin.org/image/jpeg')
if image_response.status_code == 200:
    with open('image.jpg', 'wb') as f:
        f.write(image_response.content)
    print("Изображение сохранено!")

### Модуль 3: Различные типы запросов

In [ ]:
# POST с данными формы
data = {
    'username': 'testuser',
    'password': 'testpass'
}

response = requests.post('https://httpbin.org/post', data=data)
print(f"Form Data Response: {response.json()}")

In [ ]:
# POST с JSON данными
json_data = {
    'name': 'John Doe',
    'email': 'john@example.com',
    'age': 30
}

headers = {'Content-Type': 'application/json'}
response = requests.post(
    'https://httpbin.org/post', 
    json=json_data,  # Автоматически сериализует в JSON и устанавливает заголовок
    headers=headers
)
print(f"JSON Response: {response.json()}")

In [ ]:
# Отправка файла
files = {'file': open('example.txt', 'rb')}
response = requests.post('https://httpbin.org/post', files=files)
print(f"File Upload Response: {response.json()}")

In [ ]:
# PUT запрос (обновление ресурса)
data = {'key': 'updated_value'}
response = requests.put('https://httpbin.org/put', data=data)
print(f"PUT Response: {response.json()}")

In [ ]:
response = requests.delete('https://httpbin.org/delete')

if response.status_code == 200:
    print("DELETE запрос успешно выполнен")
    print(f"Статус код: {response.status_code}")
else:
    print(f"Ошибка: {response.status_code}")

In [ ]:
# PATCH запрос (частичное обновление)
patch_data = {'email': 'new_email@example.com'}
response = requests.patch('https://httpbin.org/patch', data=patch_data)
print(f"PATCH Response: {response.json()}")

### Модуль 4: Работа с заголовками и куки

In [ ]:
# Установка заголовков
headers = {
    'User-Agent': 'MyApp/1.0',
    'Authorization': 'Bearer your_token_here',
    'Accept': 'application/json',
    'Custom-Header': 'custom_value'
}

response = requests.get('https://httpbin.org/headers', headers=headers)
print(f"Request Headers: {response.json()}")

In [ ]:
# Отправка куки с запросом
cookies = {'session_id': 'abc123', 'user_id': '456'}
response = requests.get('https://httpbin.org/cookies', cookies=cookies)
print(f"Sent Cookies: {response.json()}")

In [ ]:
# Получение куки из ответа
response = requests.get('https://httpbin.org/cookies/set?name=value')
print(f"Received Cookies: {response.cookies}")
print(f"Specific Cookie: {response.cookies.get('name')}")

In [ ]:
# Сессии для сохранения куки между запросами
session = requests.Session()
session.get('https://httpbin.org/cookies/set/sessioncookie/123456789')
response = session.get('https://httpbin.org/cookies')
print(f"Session Cookies: {response.json()}")

### Модуль 5: Обработка ошибок и таймауты

In [ ]:
try:
    response = requests.get('https://invalid-url-that-does-not-exist.com', timeout=5)
    response.raise_for_status()  # Вызывает исключение для кодов 4xx/5xx
    
except requests.exceptions.Timeout:
    print("Запрос превысил время ожидания")
    
except requests.exceptions.ConnectionError:
    print("Ошибка соединения")
    
except requests.exceptions.HTTPError as e:
    print(f"HTTP ошибка: {e}")
    
except RequestException as e:
    print(f"Общая ошибка запроса: {e}")
    
else:
    print("Запрос выполнен успешно!")

In [3]:
# Таймаут на соединение и чтение
try:
    response = requests.get('https://httpbin.org/delay/3', timeout=(2, 5))
    if response.status_code == 200:
        print("Запрос успешно выполнен!")
except requests.exceptions.Timeout:
    print("Таймаут соединения или чтения")

Запрос успешно выполнен


In [5]:
# Таймаут только на соединение
try:
    response = requests.get('https://httpbin.org/delay/1', timeout=5)
    # Общий таймаут для всего запроса
    if response.status_code == 200:
        print("Запрос успешно выполнен!") 
except requests.exceptions.Timeout:
    print("Общий таймаут запроса")

Запрос успешно выполнен!


### Модуль 6: Продвинутые техники

In [6]:
# Создание сессии для сохранения куки и заголовков между запросами
session = requests.Session()

# Установка общих заголовков для всех запросов сессии
session.headers.update({
    'User-Agent': 'MyPythonApp/1.0',
    'Accept': 'application/json'
})

# Аутентификация
auth_data = {'username': 'user', 'password': 'pass'}
session.post('https://httpbin.org/post', data=auth_data)

# Последующие запросы будут использовать куки аутентификации
response = session.get('https://httpbin.org/cookies')
print(f"Authenticated Cookies: {response.json()}")

# Закрытие сессии
session.close()

Authenticated Cookies: {'cookies': {}}


### Модуль 7: Практические примеры

Пример 1: Работа с API GitHub

In [12]:
def get_github_user_info(username):
    url = f'https://api.github.com/users/{username}'
    response = requests.get(url)
    
    if response.status_code == 200:
        user_data = response.json()
        return {
            'name': user_data.get('name'),
            'bio': user_data.get('bio'),
            'public_repos': user_data.get('public_repos'),
            'followers': user_data.get('followers')
        }
    else:
        return None

# Использование
user_info = get_github_user_info('mrvdanilov')
if user_info:
    print(f"mrvdanilov Info: {user_info}")

mrvdanilov Info: {'name': None, 'bio': None, 'public_repos': 4, 'followers': 1}


Пример 2: Загрузка файла с прогрессом

In [13]:
def download_file(url, filename):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    total_size = int(response.headers.get('content-length', 0))
    downloaded = 0
    
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
                downloaded += len(chunk)
                progress = (downloaded / total_size) * 100
                print(f"Downloaded: {progress:.1f}%", end='\r')
    
    print(f"\nFile downloaded: {filename}")

# Использование
download_file('https://httpbin.org/image/jpeg', 'downloaded_image.jpg')

Downloaded: 100.0%
File downloaded: downloaded_image.jpg


Пример 3: Пакетные запросы

In [ ]:
def fetch_url(url):
    try:
        response = requests.get(url, timeout=5)
        return {
            'url': url,
            'status': response.status_code,
            'size': len(response.content),
            'time': response.elapsed.total_seconds()
        }
    except Exception as e:
        return {
            'url': url,
            'error': str(e)
        }

def batch_requests(urls, max_workers=5):
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(fetch_url, urls))
    
    total_time = time.time() - start_time
    print(f"Total time for {len(urls)} requests: {total_time:.2f} seconds")
    
    return results

# Использование
urls = [
    'https://httpbin.org/get',
    'https://httpbin.org/ip',
    'https://httpbin.org/user-agent',
    'https://httpbin.org/headers'
]

results = batch_requests(urls)
for result in results:
    print(result)

## Заключение

Библиотека requests предоставляет мощный и удобный интерфейс для работы с HTTP-запросами в Python. Освоив эти техники, вы сможете эффективно взаимодействовать с веб-сервисами, API и другими HTTP-ресурсами.

Дополнительные ресурсы:

https://requests.readthedocs.io/en/latest/

https://www.webfx.com/web-development/glossary/http-status-codes/

https://httpbin.org/

Практикуйтесь с этими примерами и экспериментируйте с различными API, чтобы закрепить полученные знания!